In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt  


## This is a demo file for pyCCUS test drive, designed for JD

#### Step 1: have your CMG model ready as the input to pyCCUS
#### Step 2: get your parameter space ready
- This example demo alters the values of permeability, porosity, and injection rate

In [2]:
##### User input #####
# folder_path = '../data/omv/CCS/testdrive1'
folder_path = '..\\data\\omv\\CCS\\testdrive1'

params = ['phi',
          'perm_mult, md',
          'inj_rate, m3/d']

l_bounds = [0.15, 8000, 1e6]
u_bounds = [0.25, 11000, 9e6]

num_of_exp = 3
##### User input #####

In [ ]:
##### Latin Hyper Cube sampling to create the params space #####
from scipy.stats import qmc
sampler = qmc.LatinHypercube(d=len(params))
sample = sampler.random(n=num_of_exp)

sample_scaled = qmc.scale(sample, l_bounds, u_bounds)

df = pd.DataFrame(data=np.round(sample_scaled,2), columns=params)

df['perm, md'] = df['phi'] * df['perm_mult, md']

##### Save your exp design if needed #####
if not os.path.isdir(folder_path):
    os.makedirs(folder_path)
df.to_csv(os.path.join(folder_path, 'exp_design.csv'), index=False)

- Write CMG models (dat files) based on the DoE

In [3]:
df = pd.read_csv(os.path.join(folder_path, 'exp_design.csv'))
df_input = df.iloc[:,[0,2,3]]
df_input = df_input.rename(columns={'phi': 'phi', 
                                    'inj_rate, m3/d': 'inj_rate',
                                    'perm, md': 'perm'})

In [4]:
import sys
# append the path of the parent directory
sys.path.append("..")
# import method from sibling module
from utils.pyCMG_Model import omv_CCS
from utils.pySherlock import pysherlock

In [5]:
omvccs = omv_CCS()
omvccs.folder_path = folder_path
omvccs.title1 = 'pyCCUS testdrive'
omvccs.title2 = 'CCS omv'
omvccs.title3 = 'JD+YL'

In [6]:
omvccs.write_simfiles(df_input=df_input, verbose=True)

Job done -- write 3 CMG dat files based on exp design csv .....


## Run CMG files
- Option 1: local win machine
- Option 2: Stanford sherlock HPC

In [7]:
from utils.pyCMG_Control import pycmgcontrol

for nn in range(df_input.shape[0]):
    pycmg_ctrl = pycmgcontrol(exp_name=f'case{nn+1}.dat', simfolder=os.path.join(folder_path, 'datfiles'))
    # Available optoins: 'ese-win32-v2022.30', 'ese-ts1win-v2023.20', 'stf-sherlock-v2020.10', 'ese-ts2win-v2024.20'
    pycmg_ctrl.cmg_version = 'ese-ts2win-v2024.20'

    pycmg_ctrl.run_gem_simulation(case_name_suffix=f'case{nn+1}.dat')

### Option 2: run CMG on Sherlock
- You will also need submit.sh file

In [ ]:
# sherlock = pysherlock()
# pyCTRL_folder_path = os.path.join(folder_path, 'pyCTRLfiles')

# for idx in range(df_input.shape[0]):
#     sherlock.write_pyCTRLfile(folder_path=pyCTRL_folder_path, caseid=idx+1)

### Extract the result in npy format

In [7]:
from utils.pyCMG_Control import pycmgcontrol

In [8]:
for nn in range(df_input.shape[0]):
    pycmg_ctrl = pycmgcontrol(exp_name=f'case{nn+1}.dat', simfolder=os.path.join(folder_path, 'datfiles'))
    # Available optoins: 'ese-win32-v2022.30', 'ese-ts1win-v2023.20', 'stf-sherlock-v2020.10'
    pycmg_ctrl.cmg_version = 'ese-ts2win-v2024.20'
    pycmg_ctrl.rwd_precis = 4
    pycmg_ctrl.proplist = ['SG','PRES']
    pycmg_ctrl.layer_nums = [1,2,3]
    pycmg_ctrl.time_query = [2024, 2026, 2028, 2030, 2032, 2034]
    ##### Params to control rwo2npy steps ######
    pycmg_ctrl.XY2arr_interp_method = "cubic"  # options = {‘linear’, ‘nearest’, ‘cubic’}
    pycmg_ctrl.XY2arr_interp_num_x = 100
    pycmg_ctrl.XY2arr_interp_num_y = 100
    pycmg_ctrl.x_dir_key = 'X'
    pycmg_ctrl.y_dir_key = 'Y'

    pycmg_ctrl.cmgrst2npy(caseid=f"{nn+1}", verbose=False, rwodelete=False)
#     npy_data = pycmg_ctrl.cmg2npy #a nested list consisted of all data, e.g., SG&PRES together 

In [22]:
for nn in range(df_input.shape[0]):
    pycmg_ctrl = pycmgcontrol(exp_name=f'case{nn+1}.dat', simfolder=os.path.join(folder_path, 'datfiles'))
    # Available optoins: 'ese-win32-v2022.30', 'ese-ts1win-v2023.20', 'stf-sherlock-v2020.10'
    pycmg_ctrl.cmg_version = 'ese-ts2win-v2024.20'
    pycmg_ctrl.rwd_precis = 4
    pycmg_ctrl.proplist = ['Vertical Displacement from Geomechanics']
    pycmg_ctrl.layer_nums = [1,2,3]
    pycmg_ctrl.time_query = [2024, 2026, 2028, 2030, 2032, 2034]
    ##### Params to control rwo2npy steps ######
    pycmg_ctrl.XY2arr_interp_method = "cubic"  # options = {‘linear’, ‘nearest’, ‘cubic’}
    pycmg_ctrl.XY2arr_interp_num_x = 100
    pycmg_ctrl.XY2arr_interp_num_y = 100
    pycmg_ctrl.x_dir_key = 'X'
    pycmg_ctrl.y_dir_key = 'Y'

    pycmg_ctrl.cmgrst2npy(caseid=f"{nn+1}", verbose=False, rwodelete=False)
#     npy_data = pycmg_ctrl.cmg2npy #a nested list consisted of all data, e.g., SG&PRES together 

case1.gmch VERDSPLGEO has an error when reading rwo to npy ...
case2.gmch VERDSPLGEO has an error when reading rwo to npy ...
case3.gmch VERDSPLGEO has an error when reading rwo to npy ...


In [21]:
arr1=np.array(npy_data)
arr1.shape

(2, 100, 100, 3, 6)

In [10]:
l1=[i for i in range(1,12)]
l1

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

## debug code

In [ ]:
   def read_VERDSPLGEO_rwo2npy(self, case_name, save=True):
        cmgrst = pycmgresults()
        cmgrst.XY2arr_interp_method = self.XY2arr_interp_method
        cmgrst.XY2arr_interp_num_x = self.XY2arr_interp_num_x
        cmgrst.XY2arr_interp_num_y = self.XY2arr_interp_num_y

        rwo_dir = os.path.join(self.simfolder, self.batchfolder, f'rwo_{case_name}')

        ######################################################################################
        ##### No need to have this anymore ....
        # # For cases with changing injection horizon
        # if self.inj_hrzn:
        #     self.time_query = list(np.arange(self.inj_hrzn+1)+self.time_start_year)
        #     if self.yr_after_shutin_disp:
        #         for yy in self.yr_after_shutin_disp:
        #             self.time_query.append(self.inj_hrzn+self.time_start_year+yy)
        # else:
        #     print(f"Injection horizon is None, no time query for CMG result extraction ...")
        ######################################################################################

        try:

            x_new, y_new, VERDSPLGEO_arr = cmgrst.rwo_reader2arr(folder=rwo_dir,
                                                                 sim=case_name,
                                                                 prop='Vertical Displacement from Geomechanics',
                                                                 layer_nums=self.layer_nums,
                                                                 time_query=[f'Vertical Displacement from Geomechanics_{t}-Jan-01' for t in self.time_query],
                                                                 x_dir_key=self.x_dir_key, y_dir_key=self.y_dir_key)

            self.cmg2npy = VERDSPLGEO_arr
            self.cmg2npy_x_coord = x_new
            self.cmg2npy_y_coord = y_new
            
            if save == True:
                np.save(os.path.join(self.npy_folder, f"{case_name.split('.')[0]}_VERDSPLGEO.npy"), VERDSPLGEO_arr)
                return True
            else:
                return VERDSPLGEO_arr
            
        except:
            if self.err_stop:
                raise ValueError(f'{case_name} VERDSPLGEO has an error when reading rwo to npy ...')
            else:
                print(f'{case_name} VERDSPLGEO has an error when reading rwo to npy ...')